# Regularization Effects on Parallel Circuits (Fast Version)

This notebook implements experiments to test how L1, L2, and dropout regularization affect parallel circuit emergence in neural networks.

**Hypotheses:**
- **L1 regularization** → Sparsity → Fewer parallel circuits
- **L2 regularization** → Weight diffusion → More parallel circuits  
- **Dropout** → Forced redundancy → More parallel circuits

**Note:** This version removes the slow grounding/interpretation calculations and focuses only on counting circuits (neuron combinations).

---

## Setup

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = 'cuda:0'
else:
    print("Warning: CUDA not available, using CPU")
    device = 'cpu'

In [ ]:
# Clone the MI-identifiability repository
!git clone https://github.com/MelouxM/MI-identifiability.git
%cd MI-identifiability

# Install dependencies
!pip install -q -e .

## Code Modifications

We'll modify the code to:
1. Add regularization support (L1, L2, dropout)
2. Remove slow grounding/interpretation calculations
3. Keep only circuit counting (what you care about!)

In [ ]:
# Backup original files
!cp mi_identifiability/neural_model.py mi_identifiability/neural_model.py.bak
!cp main.py main.py.bak
print("Original files backed up!")

In [ ]:
# Modify main.py to remove grounding and add regularization

with open('main.py', 'r') as f:
    content = f.read()

# Step 1: Add regularization arguments
import_section = content.split('if __name__ == "__main__":')[0]
main_section = content.split('if __name__ == "__main__":')[1]

# Find where to insert new arguments
insert_pos = main_section.find("parser.add_argument('--resume-from'")

new_args = '''parser.add_argument('--l1-lambda', type=float, default=0.0,
                        help='L1 regularization coefficient (default: 0.0)')
    parser.add_argument('--l2-lambda', type=float, default=0.0,
                        help='L2 regularization coefficient (default: 0.0)')
    parser.add_argument('--dropout-rate', type=float, default=0.0,
                        help='Dropout rate, 0.0 to 1.0 (default: 0.0)')
    '''

main_section = main_section[:insert_pos] + new_args + main_section[insert_pos:]

# Step 2: Modify model creation
main_section = main_section.replace(
    'model = MLP(hidden_sizes=layer_sizes[1:-1], input_size=n_inputs, output_size=n_gates, device=args.device)',
    'model = MLP(hidden_sizes=layer_sizes[1:-1], input_size=n_inputs, output_size=n_gates, device=args.device, dropout_rate=args.dropout_rate)'
)

# Step 3: Modify training call
main_section = main_section.replace(
    '''avg_loss = model.do_train(
                x=x,
                y=y,
                x_val=x_val,
                y_val=y_val,
                batch_size=args.batch_size,
                learning_rate=lr,
                epochs=args.epochs,
                loss_target=loss_target,
                val_frequency=args.val_frequency,
                logger=logger if args.verbose else None
            )''',
    '''avg_loss = model.do_train(
                x=x,
                y=y,
                x_val=x_val,
                y_val=y_val,
                batch_size=args.batch_size,
                learning_rate=lr,
                epochs=args.epochs,
                loss_target=loss_target,
                val_frequency=args.val_frequency,
                logger=logger if args.verbose else None,
                l1_lambda=args.l1_lambda,
                l2_lambda=args.l2_lambda
            )'''
)

# Step 4: REMOVE GROUNDING SECTION
import re

# Remove grounding loop (this is the slow part!)
grounding_pattern = r'if args\.max_circuits is not None and args\.max_circuits < len\(top_circuits\):.*?print\(f\'Circuits: \{n_circuits\}, groundings: \{total_n_groundings\}\'\)'
main_section = re.sub(grounding_pattern, 
                     '# REMOVED: Grounding calculations (not needed for circuit counting)',
                     main_section, flags=re.DOTALL)

# Remove formula/mapping section (also slow!)
formula_pattern = r'formulas = set\(\).*?print\(f\'Formulas: \{n_formulas\}, mappings: \{total_n_mappings\}\'\)'
main_section = re.sub(formula_pattern,
                     '# REMOVED: Formula/mapping calculations (not needed for circuit counting)',
                     main_section, flags=re.DOTALL)

# Step 5: Simplify data storage (remove interpretation fields)
main_section = main_section.replace(
    "'interpretations_per_circuit': total_n_groundings,",
    "# 'interpretations_per_circuit' removed (not needed)"
)
main_section = main_section.replace(
    "'formulas': n_formulas,",
    "# 'formulas' removed (not needed)"
)
main_section = main_section.replace(
    "'mappings_per_formula': total_n_mappings,",
    "# 'mappings_per_formula' removed (not needed)"
)

# Step 6: Add regularization info to data
main_section = main_section.replace(
    "'weights': weights,",
    ''''weights': weights,
                'l1_lambda': args.l1_lambda,
                'l2_lambda': args.l2_lambda,
                'dropout_rate': args.dropout_rate'''
)

# Write modified main.py
with open('main.py', 'w') as f:
    f.write(import_section + 'if __name__ == "__main__":' + main_section)

print("✓ main.py modified successfully!")
print("  - Added regularization support")
print("  - Removed slow grounding calculations")
print("  - Removed formula/mapping calculations")
print("  - Kept circuit counting (what you need!)")

In [ ]:
# Modify neural_model.py to add dropout and regularization
# (This is the same as before - adds dropout support and L1/L2 regularization)

with open('mi_identifiability/neural_model.py', 'r') as f:
    content = f.read()

# Add dropout_rate parameter to __init__
content = content.replace(
    'def __init__(self, hidden_sizes: list, input_size=2, output_size=1, activation=\'leaky_relu\', device=\'cpu\'):',
    'def __init__(self, hidden_sizes: list, input_size=2, output_size=1, activation=\'leaky_relu\', device=\'cpu\', dropout_rate=0.0):'
)

# Store dropout_rate
content = content.replace(
    'self.activation = activation',
    'self.activation = activation\n        self.dropout_rate = dropout_rate'
)

# Modify layer creation to add dropout
old_layers = '''self.layers = nn.ModuleList(
            nn.Sequential(
                nn.Linear(in_size, out_size),
                ACTIVATION_FUNCTIONS[activation]() if idx < len(self.layer_sizes) - 1 else nn.Identity()
            ) for idx, (in_size, out_size) in enumerate(zip(self.layer_sizes, self.layer_sizes[1:]))
        )'''

new_layers = '''# Build layers with optional dropout
        self.layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layer_sizes, self.layer_sizes[1:])):
            layer_components = [nn.Linear(in_size, out_size)]
            
            # Add dropout after linear layer (but not on output layer)
            if idx < len(self.layer_sizes) - 2 and dropout_rate > 0:
                layer_components.append(nn.Dropout(dropout_rate))
            
            # Add activation function (Identity for output layer)
            if idx < len(self.layer_sizes) - 2:
                layer_components.append(ACTIVATION_FUNCTIONS[activation]())
            else:
                layer_components.append(nn.Identity())
            
            self.layers.append(nn.Sequential(*layer_components))'''

content = content.replace(old_layers, new_layers)

# Add dropout_rate to save
content = content.replace(
    "'activation': self.activation,",
    "'activation': self.activation,\n            'dropout_rate': self.dropout_rate,"
)

# Add dropout_rate to load (with backwards compatibility)
content = content.replace(
    "activation=model_data['activation']",
    "activation=model_data['activation'],\n            dropout_rate=model_data.get('dropout_rate', 0.0)"
)

# Add L1 and L2 regularization to do_train
content = content.replace(
    'def do_train(self, x, y, x_val, y_val, batch_size, learning_rate, epochs, loss_target=0.001, val_frequency=10,\n                 early_stopping_steps=3, logger=None):',
    'def do_train(self, x, y, x_val, y_val, batch_size, learning_rate, epochs, loss_target=0.001, val_frequency=10,\n                 early_stopping_steps=3, logger=None, l1_lambda=0.0, l2_lambda=0.0):'
)

# Add regularization to loss calculation
old_loss = '''for inputs, targets in dataloader:
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                loss.backward()'''

new_loss = '''for inputs, targets in dataloader:
                optimizer.zero_grad()
                outputs = self(inputs)
                
                # Base loss
                base_loss = criterion(outputs, targets)
                loss = base_loss
                
                # Add L1 Regularization
                if l1_lambda > 0:
                    l1_penalty = sum(torch.sum(torch.abs(param)) for param in self.parameters())
                    loss = loss + l1_lambda * l1_penalty
                
                # Add L2 Regularization
                if l2_lambda > 0:
                    l2_penalty = sum(torch.sum(param ** 2) for param in self.parameters())
                    loss = loss + l2_lambda * l2_penalty
                
                loss.backward()'''

content = content.replace(old_loss, new_loss)

with open('mi_identifiability/neural_model.py', 'w') as f:
    f.write(content)

print("✓ neural_model.py modified successfully!")
print("  - Added dropout support")
print("  - Added L1/L2 regularization")

## Quick Test

Let's verify everything works with a small test.

In [ ]:
# Quick test run with baseline (no regularization)
!python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
  --n-experiments 3 --size 3 --depth 2 --device {device}

print("\n" + "="*60)
print("✓ Test run completed successfully!")
print("✓ Ready to run full experiments (much faster without grounding!)")
print("="*60)

## Run Full Experiments

Now we'll run the complete experimental suite. Each section can be run independently.

In [ ]:
# Baseline - No regularization
print("Running baseline experiments...")
!python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
  --n-experiments 50 --size 3 --depth 2 --device {device}

print("\n✓ Baseline completed!")

In [ ]:
# L1 Regularization Experiments
l1_lambdas = [0.0001, 0.0005, 0.001, 0.005, 0.01]

for l1 in l1_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L1 experiment: lambda={l1}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 50 --size 3 --depth 2 --device {device} --l1-lambda {l1}

print("\n✓ All L1 experiments completed!")

In [ ]:
# L2 Regularization Experiments
l2_lambdas = [0.0001, 0.0005, 0.001, 0.005, 0.01]

for l2 in l2_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L2 experiment: lambda={l2}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 50 --size 3 --depth 2 --device {device} --l2-lambda {l2}

print("\n✓ All L2 experiments completed!")

In [ ]:
# Dropout Experiments
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]

for dr in dropout_rates:
    print(f"\n{'='*60}")
    print(f"Running Dropout experiment: rate={dr}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 50 --size 3 --depth 2 --device {device} --dropout-rate {dr}

print("\n✓ All Dropout experiments completed!")

## Analysis

Now let's analyze the results and test our hypotheses.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import ast

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Load all results
results_dir = Path('logs')
all_data = []

for csv_file in results_dir.glob("**/df_out.csv"):
    df = pd.read_csv(csv_file)
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)
print(f"Loaded {len(combined_df)} experiment results")

# Extract circuit counts from lists
def extract_first(x):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except:
            return x
    if isinstance(x, list) and len(x) > 0:
        return x[0]
    return x

combined_df['n_circuits'] = combined_df['perfect_circuits'].apply(extract_first)

# Ensure regularization columns exist
for col in ['l1_lambda', 'l2_lambda', 'dropout_rate']:
    if col not in combined_df.columns:
        combined_df[col] = 0.0

print(f"\nCircuit count summary:")
print(combined_df['n_circuits'].describe())
print(f"\nRegularization conditions tested:")
print(f"  L1 lambdas: {sorted(combined_df['l1_lambda'].unique())}")
print(f"  L2 lambdas: {sorted(combined_df['l2_lambda'].unique())}")
print(f"  Dropout rates: {sorted(combined_df['dropout_rate'].unique())}")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Regularization Effects on Parallel Circuits', fontsize=16, fontweight='bold')

# L1 - Box plot
if combined_df['l1_lambda'].nunique() > 1:
    sns.boxplot(data=combined_df, x='l1_lambda', y='n_circuits', ax=axes[0, 0])
    axes[0, 0].set_title('L1: Circuit Count Distribution', fontweight='bold')
    axes[0, 0].set_xlabel('L1 Lambda')
    axes[0, 0].set_ylabel('Number of Circuits')

# L1 - Mean trend
if combined_df['l1_lambda'].nunique() > 1:
    mean_circuits = combined_df.groupby('l1_lambda')['n_circuits'].mean()
    std_circuits = combined_df.groupby('l1_lambda')['n_circuits'].std()
    axes[1, 0].errorbar(mean_circuits.index, mean_circuits.values, 
                       yerr=std_circuits.values, marker='o', linewidth=2, 
                       markersize=8, capsize=5)
    axes[1, 0].set_title('L1: Mean Circuit Count ± SD', fontweight='bold')
    axes[1, 0].set_xlabel('L1 Lambda')
    axes[1, 0].set_ylabel('Mean Circuit Count')
    axes[1, 0].grid(True, alpha=0.3)

# L2 - Box plot
if combined_df['l2_lambda'].nunique() > 1:
    sns.boxplot(data=combined_df, x='l2_lambda', y='n_circuits', ax=axes[0, 1], color='orange')
    axes[0, 1].set_title('L2: Circuit Count Distribution', fontweight='bold')
    axes[0, 1].set_xlabel('L2 Lambda')
    axes[0, 1].set_ylabel('Number of Circuits')

# L2 - Mean trend
if combined_df['l2_lambda'].nunique() > 1:
    mean_circuits = combined_df.groupby('l2_lambda')['n_circuits'].mean()
    std_circuits = combined_df.groupby('l2_lambda')['n_circuits'].std()
    axes[1, 1].errorbar(mean_circuits.index, mean_circuits.values,
                       yerr=std_circuits.values, marker='o', linewidth=2,
                       markersize=8, capsize=5, color='orange')
    axes[1, 1].set_title('L2: Mean Circuit Count ± SD', fontweight='bold')
    axes[1, 1].set_xlabel('L2 Lambda')
    axes[1, 1].set_ylabel('Mean Circuit Count')
    axes[1, 1].grid(True, alpha=0.3)

# Dropout - Box plot
if combined_df['dropout_rate'].nunique() > 1:
    sns.boxplot(data=combined_df, x='dropout_rate', y='n_circuits', ax=axes[0, 2], color='green')
    axes[0, 2].set_title('Dropout: Circuit Count Distribution', fontweight='bold')
    axes[0, 2].set_xlabel('Dropout Rate')
    axes[0, 2].set_ylabel('Number of Circuits')

# Dropout - Mean trend
if combined_df['dropout_rate'].nunique() > 1:
    mean_circuits = combined_df.groupby('dropout_rate')['n_circuits'].mean()
    std_circuits = combined_df.groupby('dropout_rate')['n_circuits'].std()
    axes[1, 2].errorbar(mean_circuits.index, mean_circuits.values,
                       yerr=std_circuits.values, marker='o', linewidth=2,
                       markersize=8, capsize=5, color='green')
    axes[1, 2].set_title('Dropout: Mean Circuit Count ± SD', fontweight='bold')
    axes[1, 2].set_xlabel('Dropout Rate')
    axes[1, 2].set_ylabel('Mean Circuit Count')
    axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('all_regularization_effects.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comprehensive plot saved as 'all_regularization_effects.png'")

In [ ]:
# Statistical Analysis
print("\n" + "="*70)
print(" "*15 + "HYPOTHESIS TESTING RESULTS")
print("="*70 + "\n")

# Get baseline
baseline = combined_df[
    (combined_df['l1_lambda'] == 0) & 
    (combined_df['l2_lambda'] == 0) & 
    (combined_df['dropout_rate'] == 0)
]['n_circuits']

baseline_mean = baseline.mean()
baseline_std = baseline.std()

print(f"BASELINE (No Regularization):")
print(f"  Mean: {baseline_mean:.2f} ± {baseline_std:.2f}")
print(f"  N = {len(baseline)}")
print()

def test_hypothesis(df, col_name, col_value, baseline, hypothesis_direction):
    """Test if regularization effect matches hypothesis"""
    treatment = df[df[col_name] == col_value]['n_circuits']
    
    if len(treatment) == 0:
        return None
    
    # t-test
    t_stat, p_val = stats.ttest_ind(baseline, treatment)
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((baseline.std()**2 + treatment.std()**2) / 2)
    cohens_d = (treatment.mean() - baseline.mean()) / pooled_std
    
    # Mean change
    mean_change = treatment.mean() - baseline.mean()
    pct_change = 100 * mean_change / baseline.mean()
    
    # Check if direction matches hypothesis
    if hypothesis_direction == 'decrease':
        direction_correct = mean_change < 0
    else:  # 'increase'
        direction_correct = mean_change > 0
    
    significant = p_val < 0.05
    supported = direction_correct and significant
    
    return {
        'mean_change': mean_change,
        'pct_change': pct_change,
        'p_value': p_val,
        'cohens_d': cohens_d,
        'n': len(treatment),
        'direction_correct': direction_correct,
        'significant': significant,
        'supported': supported
    }

# Test L1 hypothesis
print("="*70)
print("L1 REGULARIZATION (Hypothesis: Sparsity → Fewer Circuits)")
print("="*70)
if combined_df['l1_lambda'].nunique() > 1:
    for l1 in sorted(combined_df[combined_df['l1_lambda'] > 0]['l1_lambda'].unique()):
        result = test_hypothesis(combined_df, 'l1_lambda', l1, baseline, 'decrease')
        if result:
            print(f"\nL1 = {l1}:")
            print(f"  Mean change: {result['mean_change']:+.2f} circuits ({result['pct_change']:+.1f}%)")
            print(f"  Cohen's d: {result['cohens_d']:.3f}")
            print(f"  p-value: {result['p_value']:.4f}")
            print(f"  Direction: {'✓ Correct (decrease)' if result['direction_correct'] else '✗ Wrong (increase)'}")
            print(f"  Significant: {'✓ Yes' if result['significant'] else '✗ No'}")
            print(f"  Hypothesis: {'✓✓ SUPPORTED' if result['supported'] else '✗✗ NOT SUPPORTED'}")
print()

# Test L2 hypothesis
print("="*70)
print("L2 REGULARIZATION (Hypothesis: Diffusion → More Circuits)")
print("="*70)
if combined_df['l2_lambda'].nunique() > 1:
    for l2 in sorted(combined_df[combined_df['l2_lambda'] > 0]['l2_lambda'].unique()):
        result = test_hypothesis(combined_df, 'l2_lambda', l2, baseline, 'increase')
        if result:
            print(f"\nL2 = {l2}:")
            print(f"  Mean change: {result['mean_change']:+.2f} circuits ({result['pct_change']:+.1f}%)")
            print(f"  Cohen's d: {result['cohens_d']:.3f}")
            print(f"  p-value: {result['p_value']:.4f}")
            print(f"  Direction: {'✓ Correct (increase)' if result['direction_correct'] else '✗ Wrong (decrease)'}")
            print(f"  Significant: {'✓ Yes' if result['significant'] else '✗ No'}")
            print(f"  Hypothesis: {'✓✓ SUPPORTED' if result['supported'] else '✗✗ NOT SUPPORTED'}")
print()

# Test Dropout hypothesis
print("="*70)
print("DROPOUT (Hypothesis: Redundancy → More Circuits)")
print("="*70)
if combined_df['dropout_rate'].nunique() > 1:
    for dr in sorted(combined_df[combined_df['dropout_rate'] > 0]['dropout_rate'].unique()):
        result = test_hypothesis(combined_df, 'dropout_rate', dr, baseline, 'increase')
        if result:
            print(f"\nDropout = {dr}:")
            print(f"  Mean change: {result['mean_change']:+.2f} circuits ({result['pct_change']:+.1f}%)")
            print(f"  Cohen's d: {result['cohens_d']:.3f}")
            print(f"  p-value: {result['p_value']:.4f}")
            print(f"  Direction: {'✓ Correct (increase)' if result['direction_correct'] else '✗ Wrong (decrease)'}")
            print(f"  Significant: {'✓ Yes' if result['significant'] else '✗ No'}")
            print(f"  Hypothesis: {'✓✓ SUPPORTED' if result['supported'] else '✗✗ NOT SUPPORTED'}")

print("\n" + "="*70)

In [ ]:
# Export results
combined_df.to_csv('all_results.csv', index=False)
print("✓ Results saved to 'all_results.csv'")

# Create summary report
with open('hypothesis_summary.txt', 'w') as f:
    f.write("REGULARIZATION EFFECTS ON PARALLEL CIRCUITS\n")
    f.write("="*70 + "\n\n")
    
    f.write(f"Baseline: {baseline_mean:.2f} ± {baseline_std:.2f} circuits\n\n")
    
    # Summarize each hypothesis
    for reg_type, col, direction in [('L1', 'l1_lambda', 'decrease'),
                                      ('L2', 'l2_lambda', 'increase'),
                                      ('Dropout', 'dropout_rate', 'increase')]:
        f.write(f"\n{reg_type} Hypothesis ({direction}):\n")
        f.write("-" * 40 + "\n")
        
        if combined_df[col].nunique() > 1:
            max_val = combined_df[combined_df[col] > 0][col].max()
            result = test_hypothesis(combined_df, col, max_val, baseline, direction)
            
            if result:
                f.write(f"Strongest effect at {col}={max_val}:\n")
                f.write(f"  Change: {result['mean_change']:+.2f} ({result['pct_change']:+.1f}%)\n")
                f.write(f"  Effect size: {result['cohens_d']:.3f}\n")
                f.write(f"  p-value: {result['p_value']:.4f}\n")
                f.write(f"  Status: {'SUPPORTED' if result['supported'] else 'NOT SUPPORTED'}\n")

print("✓ Summary saved to 'hypothesis_summary.txt'")

# Zip everything
!zip -r regularization_results.zip logs/ *.png *.csv *.txt

print("\n✓ All results packaged!")
print("\nDownloading results...")

from google.colab import files
files.download('regularization_results.zip')

print("\n" + "="*70)
print("EXPERIMENT COMPLETE!")
print("="*70)